December 4, 2024

Goal: Perform the first set of experiments doing equivalence checkings on benchmark circuits from feynman.

In [1]:
cd ..

/home/abhishekabhishek/git/cpsc-513-project


First, let's load the benchmark names from the .csv file using pd.

In [2]:
import pandas as pd
circ_info_df = pd.read_csv('feynman_benchmark_properties.csv')
circ_info_df = circ_info_df.sort_values(by='n_gates_original')
circ_info_df

,Unnamed: 0,name,n_gates_original,n_gates_transpiled,depth_original,depth_transpiled,gate_set,n_qubits_original,n_qubits_transpiled
0,13,tof_3,15,182,11,116,h_ccx,5,127
1,29,barenco_tof_3,20,282,14,159,h_ccx,5,127
3,36,mod5_4,23,283,23,179,h_ccx_cx_x,5,127
6,9,tof_4,25,334,17,191,h_ccx,7,127
4,10,barenco_tof_4,34,577,26,308,h_ccx,7,127
9,18,tof_5,35,494,23,276,h_ccx,9,127
7,7,mod_mult_55,49,610,20,246,h_x_ccx_cx,9,127
8,19,barenco_tof_5,50,969,38,532,h_ccx,9,127
11,22,vbe_adder_3,50,872,28,478,h_ccx_cx,10,127
13,24,gf2^4_mult,65,1454,31,662,h_ccx_cx,12,127


In [3]:
benchmark_names = circ_info_df.name
benchmark_names = benchmark_names.to_list()

Let's setup the logging:

In [4]:
import logging
import sys

from pythonjsonlogger import jsonlogger
logger = logging.getLogger()

stdout_handler = logging.StreamHandler(stream=sys.stdout)
file_handler = logging.FileHandler('feynman_eq_checking_results.txt')

json_fmt = jsonlogger.JsonFormatter(
    "%(name)s %(asctime)s %(levelname)s %(filename)s %(lineno)s %(process)d %(message)s",
    rename_fields={"levelname": "severity", "asctime": "timestamp"},
    datefmt="%Y-%m-%dT%H:%M:%SZ",
)

stdout_handler.setFormatter(json_fmt)
file_handler.setFormatter(json_fmt)

logger.addHandler(stdout_handler)
logger.addHandler(file_handler)

Let's setup the config for `qcec.verify()`:

In [4]:
from mqt import qcec
from mqt.qcec import Configuration
from mqt.qcec.configuration import augment_config_from_kwargs

**NOTE: first, let's try out a smaller timeout to see how far along the benchmarks can we push the equivalence checking**

In [5]:
config = Configuration()
config_dict = {
    # application
    "alternating_scheme": "proportional",
    "simulation_scheme": "proportional",
    
    # execution
    "run_zx_checker": False,
    "timeout": 60.,
    
    # functionality
    "check_partial_equivalence": False,
    
    # optimizations
    "elide_permutations": False,
    "fuse_single_qubit_gates": False,
    "reconstruct_swaps": False,
    "reorder_operations": False,
    "transform_dynamic_circuit": False,
    
    # simulation
}

# update the configuration with the dictionary
augment_config_from_kwargs(config, config_dict)
config

{
  "application": {
    "alternating": "proportional",
    "simulation": "proportional"
  },
  "execution": {
    "nthreads": 20,
    "parallel": true,
    "run_alternating_checker": true,
    "run_construction_checker": false,
    "run_simulation_checker": true,
    "run_zx_checker": false,
    "timeout": 60.0,
    "tolerance": 2.2737367544323206e-13
  },
  "functionality": {
    "check_partial_equivalence": false,
    "trace_threshold": 1e-08
  },
  "optimizations": {
    "backpropagate_output_permutation": false,
    "elide_permutations": false,
    "fuse_consecutive_single_qubit_gates": false,
    "reconstruct_swaps": false,
    "remove_diagonal_gates_before_measure": false,
    "reorder_operations": false,
    "transform_dynamic_circuit": false
  },
  "parameterized": {
    "additional_instantiations": 0,
    "tolerance": 1e-12
  },
  "simulation": {
    "fidelity_threshold": 1e-08,
    "max_sims": 18,
    "seed": 0,
    "state_type": "computational_basis",
    "store_counterexam

Let's perform the eq. checking by compiling the circuits to `ibm_sherbrooke`:

First, let's setup the backend:

In [2]:
from keys import IBMQ_API
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", token=IBMQ_API, instance="ibm-q/open/main")
ibm_backend = service.backend("ibm_sherbrooke")

In [3]:
type(ibm_backend)

qiskit_ibm_runtime.ibm_backend.IBMBackend

Now, let's run the experiments:

In [7]:
from tqdm import tqdm
from timeit import default_timer as timer
from qiskit import QuantumCircuit

In [8]:
benchmark_names = ['ham15-high']

In [ ]:
for benchmark_name in tqdm(benchmark_names):
    # load the qasm file
    qasm_path = "../feynman/benchmarks/qasm/" + benchmark_name + ".qasm"
    circ = QuantumCircuit.from_qasm_file(qasm_path)

    # IMPORTANT: add measurements to the end of the circuit
    circ.measure_all()

    # transpile to a HW backend
    circ_hw = transpile(circ, ibm_backend)

    # verify equivalence
    result = qcec.verify(circ, circ_hw, config)

    # log the times and the results
    log_string = f'name: {benchmark_name}, preprocessing: {result.preprocessing_time}' +\
        f', check: {result.check_time}, equivalence: {result.equivalence}'
    print(log_string)

  0%|          | 0/1 [00:00<?, ?it/s]

I think the units for the time above are in seconds!